In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
# Export model
def export_onnx(model, data_tensor_array, model_loc):
  circuit = model()

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  # print(device)

  circuit.to(device)

  # Flips the neural net into inference mode
  circuit.eval()
  input_names = []
  dynamic_axes = {}

  data_tensor_tuple = ()
  for i in range(len(data_tensor_array)):
    data_tensor_tuple += (data_tensor_array[i],)
    input_index = "input"+str(i+1)
    input_names.append(input_index)
    dynamic_axes[input_index] = {0 : 'batch_size'}
  dynamic_axes["output"] = {0 : 'batch_size'}

  # Export the model
  torch.onnx.export(circuit,               # model being run
                      data_tensor_tuple,                   # model input (or a tuple for multiple inputs)
                      model_loc,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=11,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = input_names,   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes=dynamic_axes)

In [4]:
# mode is either "accuracy" or "resources"

def gen_settings(comb_data_path, onnx_filename, scale, mode, settings_filename):
  print("==== Generate & Calibrate Setting ====")
  # Set input to be Poseidon Hash, and param of computation graph to be public
  # Poseidon is not homomorphic additive, maybe consider Pedersens or Dory commitment.
  gip_run_args = ezkl.PyRunArgs()
  gip_run_args.input_visibility = "hashed"  # matrix and generalized inverse commitments
  gip_run_args.output_visibility = "public"   # no parameters used
  gip_run_args.param_visibility = "private" # should be Tensor(True)--> to enforce arbitrary data in w

 # generate settings
  ezkl.gen_settings(onnx_filename, settings_filename, py_run_args=gip_run_args)
  if scale =="default":
    ezkl.calibrate_settings(
    comb_data_path, onnx_filename, settings_filename, mode)
  else:
    ezkl.calibrate_settings(
    comb_data_path, onnx_filename, settings_filename, mode, scales = scale)

  assert os.path.exists(settings_filename)
  assert os.path.exists(comb_data_path)
  assert os.path.exists(onnx_filename)
  f_setting = open(settings_filename, "r")
  print("scale: ", scale)
  print("setting: ", f_setting.read())


In [5]:
def verifier_init(verifier_model, verifier_model_path, verifier_compiled_model_path, dummy_data_path_array, settings_path, srs_path, pk_path, vk_path, scale, mode):

  # load data from dummy_data_path_array into dummy_data_array
  dummy_data_tensor_array = []
  comb_dummy_data = []
  for path in dummy_data_path_array:
    dummy_data = np.array(json.loads(open(path, "r").read())["input_data"][0])
    # print("dumm: ", dummy_data)
    dummy_data_tensor_array.append(torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 )))
    comb_dummy_data.append(dummy_data.tolist())
  # export onnx file
  export_onnx(verifier_model,dummy_data_tensor_array, verifier_model_path)

  comb_dummy_data_path = os.path.join('generated/comb_dummy_data.json')
  # Serialize data into file:
  json.dump(dict(input_data = comb_dummy_data), open(comb_dummy_data_path, 'w' ))

  # gen + calibrate setting
  gen_settings(comb_dummy_data_path, verifier_model_path, scale, mode, settings_path)

  # compile circuit
  res = ezkl.compile_circuit(verifier_model_path, verifier_compiled_model_path, settings_path)
  assert res == True

  # srs path
  res = ezkl.get_srs(srs_path, settings_path)

  # setupt vk, pk param for use..... prover can use same pk or can init their own!
  print("==== setting up ezkl ====")
  start_time = time.time()
  res = ezkl.setup(
        verifier_compiled_model_path,
        vk_path,
        pk_path,
        srs_path)
  end_time = time.time()
  time_setup = end_time -start_time
  print(f"Time setup: {time_setup} seconds")

  assert res == True
  assert os.path.isfile(vk_path)
  assert os.path.isfile(pk_path)
  assert os.path.isfile(settings_path)

In [6]:
def prover_gen(prover_model, data_path_array, witness_path, prover_model_path, prover_compiled_model_path, settings_path, proof_path):
  # load data from data_path
  # data = json.loads(open(data_path, "r").read())["input_data"][0]
  # data_tensor = torch.reshape(torch.tensor(data), (1, len(data),1 ))


  data_tensor_array = []
  comb_data = []
  for path in data_path_array:
    data = np.array(json.loads(open(path, "r").read())["input_data"][0])
    # print("dumm: ", dummy_data)
    data_tensor_array.append(torch.reshape(torch.tensor(data), (1, len(data),1 )))
    comb_data.append(data.tolist())

  # export onnx file
  export_onnx(prover_model, data_tensor_array, prover_model_path)

  comb_data_path = os.path.join('generated/comb_data.json')
  # Serialize data into file:
  json.dump(dict(input_data = comb_data), open(comb_data_path, 'w' ))

  res = ezkl.compile_circuit(prover_model_path, prover_compiled_model_path, settings_path)
  assert res == True
  # now generate the witness file
  print('==== Generating Witness ====')
  witness = ezkl.gen_witness(comb_data_path, prover_compiled_model_path, witness_path)
  assert os.path.isfile(witness_path)
  # print(witness["outputs"])
  settings = json.load(open(settings_path))
  output_scale = settings['model_output_scales']
  print("witness boolean: ", ezkl.vecu64_to_float(witness['outputs'][0][0], output_scale[0]))
  for i in range(len(witness['outputs'][1])):
    print("witness result", i+1,":", ezkl.vecu64_to_float(witness['outputs'][1][i], output_scale[1]))

  # GENERATE A PROOF
  print("==== Generating Proof ====")
  start_time = time.time()
  res = ezkl.prove(
        witness_path,
        prover_compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "single",
    )

  print("proof: " ,res)
  end_time = time.time()
  time_gen_prf = end_time -start_time
  print(f"Time gen prf: {time_gen_prf} seconds")
  assert os.path.isfile(proof_path)

In [7]:
def verifier_verify(proof_path, settings_path, vk_path, srs_path):
  # enforce boolean statement to be true
  settings = json.load(open(settings_path))
  output_scale = settings['model_output_scales']

  proof = json.load(open(proof_path))
  num_inputs = len(settings['model_input_scales'])
  print("num_inputs: ", num_inputs)
  proof["instances"][0][num_inputs] = ezkl.float_to_vecu64(1.0, output_scale[0])
  json.dump(proof, open(proof_path, 'w'))

  print("prf instances: ", proof['instances'])

  print("proof boolean: ", ezkl.vecu64_to_float(proof['instances'][0][num_inputs], output_scale[0]))
  for i in range(num_inputs+1, len(proof['instances'][0])):
    print("proof result",i-num_inputs,":", ezkl.vecu64_to_float(proof['instances'][0][i], output_scale[1]))


  res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

  assert res == True
  print("verified")

In [9]:
# init path
os.makedirs(os.path.dirname('generated/'), exist_ok=True)
verifier_model_path = os.path.join('generated/verifier.onnx')
prover_model_path = os.path.join('generated/prover.onnx')
verifier_compiled_model_path = os.path.join('generated/verifier.compiled')
prover_compiled_model_path = os.path.join('generated/prover.compiled')
pk_path = os.path.join('generated/test.pk')
vk_path = os.path.join('generated/test.vk')
proof_path = os.path.join('generated/test.pf')
settings_path = os.path.join('generated/settings.json')
srs_path = os.path.join('generated/kzg.srs')
witness_path = os.path.join('generated/witness.json')

=======================  ZK-STATS FLOW =======================

In [10]:
x_vals_path = os.path.join('x_vals.json')
dummy_x_vals_path = os.path.join('generated/dummy_x_vals.json')
x_open = open(x_vals_path, "r")
x_vals= json.loads(x_open.read())['input_data'][0]
dummy_x_vals = np.random.uniform(min(x_vals), max(x_vals), len(x_vals))
json.dump({"input_data":[dummy_x_vals.tolist()]}, open(dummy_x_vals_path, 'w'))


y_vals_path = os.path.join('y_vals.json')
dummy_y_vals_path = os.path.join('generated/dummy_y_vals.json')
y_open = open(y_vals_path, "r")
y_vals= json.loads(y_open.read())["input_data"][0]
dummy_y_vals = np.random.uniform(min(y_vals), max(y_vals), len(y_vals))
json.dump({"input_data":[dummy_y_vals.tolist()]}, open(dummy_y_vals_path, 'w'))


real_cov = statistics.covariance(x_vals, y_vals)
x_mean = statistics.mean(x_vals)
y_mean = statistics.mean(y_vals)
print("cov: ",real_cov )
print("x mean: ", x_mean)
print("y mean: ", y_mean)

dummy_cov = statistics.covariance(dummy_x_vals, dummy_y_vals)
dummy_x_mean = statistics.mean(dummy_x_vals)
dummy_y_mean = statistics.mean(dummy_y_vals)

cov:  3214.750736446082
x mean:  49.5
y mean:  227.70285322096493


In [11]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.cov = nn.Parameter(data = torch.tensor(dummy_cov), requires_grad = False)
        self.x_mean = nn.Parameter(data = torch.tensor(dummy_x_mean), requires_grad = False)
        self.y_mean = nn.Parameter(data = torch.tensor(dummy_y_mean), requires_grad = False)
    def forward(self,X,Y):
        # some expression of tolerance to error in the inference
        # print("x size: ", X.size()[1])
        #  need to enforce same length, not yet
        return (torch.logical_and(torch.logical_and(torch.abs(torch.sum(X)-X.size()[1]*(self.x_mean))<0.01*torch.sum(X), torch.abs(torch.sum(Y)-Y.size()[1]*(self.y_mean))<0.01*torch.sum(Y)), torch.abs(torch.sum((X-self.x_mean)*(Y-self.y_mean))-(X.size()[1]-1)*(self.cov))<0.01*(X.size()[1]-1)*(self.cov)), self.cov)

verifier_init(verifier_model, verifier_model_path, verifier_compiled_model_path, [dummy_x_vals_path, dummy_y_vals_path], settings_path, srs_path, pk_path, vk_path,[0], "resources")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-4,65276],"logrows":16,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":13120,"total_assignments":824,"total_const_size":2,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0,0],"module_sizes":{"kzg":[],"poseidon":[13120,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}
==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 5.766196966171265 seconds


In [12]:
# Prover/ data owner side
theory_output = torch.tensor(real_cov)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.cov = nn.Parameter(data = torch.tensor(real_cov), requires_grad = False)
        self.x_mean = nn.Parameter(data = torch.tensor(x_mean), requires_grad = False)
        self.y_mean = nn.Parameter(data = torch.tensor(y_mean), requires_grad = False)
    def forward(self,X,Y):
        # some expression of tolerance to error in the inference
        # print("x size: ", X.size()[1])
        #  need to enforce same length
        return (torch.logical_and(torch.logical_and(torch.abs(torch.sum(X)-X.size()[1]*(self.x_mean))<0.01*torch.sum(X), torch.abs(torch.sum(Y)-Y.size()[1]*(self.y_mean))<0.01*torch.sum(Y)), torch.abs(torch.sum((X-self.x_mean)*(Y-self.y_mean))-(X.size()[1]-1)*(self.cov))<0.01*(X.size()[1]-1)*(self.cov)), self.cov)

prover_gen(prover_model, [x_vals_path, y_vals_path], witness_path, prover_model_path, prover_compiled_model_path, settings_path, proof_path)

==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 3215.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[14955570959218682635, 4667139652385906200, 12836539004462631467, 1774684518626433649], [2636517083442646612, 5059672846326347070, 4705835036930416229, 3312472395716484608], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [5743885005642251665, 3430503978676436355, 7149667244725939006, 2902673458086333540]]], 'proof': '1acc73a4594b0b898d30afd10dd4082a55f3010e42bf99a0bcade24d2d5ae4112f418d0e09a957b8d175ba5ce95337ee537a160d52926db85318d49748a169f32b28c3e6788fccdc97a7b1688a467de6e8fe2b984dc5aecc407489d9fb653944050693f80ae110f831614f87a163514062efe31cf2033dbc708f904ad4a1f41027de0b5dbd6d4e4db487b35747844955a22608d138003609ea5b68344d0c011413fd992c7bea5ed5e072fda0b09d59b548427b04cbb9ebfab7714f0d029f100d181f7bf0d1e6863d357b47c37aa37bf61090ef3e895647711ea99fa38c23cd8b19b849bad1b225ab42ec2f7260b84566071dd96a6b6f07802174704d28d1b45e014f5bc7e235d400341f357bb0f8a0cda6b3835e577fde1c95f9cfe64e18fb631af362c316b44f9441a88c096ca26fe151a5bb3ce

In [13]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  2
prf instances:  [[[14955570959218682635, 4667139652385906200, 12836539004462631467, 1774684518626433649], [2636517083442646612, 5059672846326347070, 4705835036930416229, 3312472395716484608], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [5743885005642251665, 3430503978676436355, 7149667244725939006, 2902673458086333540]]]
proof boolean:  1.0
proof result 1 : 3215.0
verified
